# Worflow:
- Read dataset.
- Preprocessing.
- Splitting the dataset.
- Training a Decision Tree model & Random forest.
- Choose one mode for deployment.
- Creating persistent model.

**We have a dataset that has feautres of cars and their condition, we are building and deploying a classifier that predict the car condition**

In [13]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier


data = pd.read_csv('car.data.txt', names=['Buying', 'Maint','doors', 'persons','lug-boot','safety','target'])
data.head()

,Buying,Maint,doors,persons,lug-boot,safety,target
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [14]:
# target is our label column, let's check it out
data['target'].unique()

array(['unacc', 'acc', 'vgood', 'good'], dtype=object)

**As you see the car condition has ne of the four categorise**

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
Buying      1728 non-null object
Maint       1728 non-null object
doors       1728 non-null object
persons     1728 non-null object
lug-boot    1728 non-null object
safety      1728 non-null object
target      1728 non-null object
dtypes: object(7)
memory usage: 94.6+ KB


**Most of the columns are categorical or have string data type, we need to transform them into numeric**

In [16]:
# tranformation functions
def Trans_Buying(x):
    
    if x== 'vhigh': 
        return 0
    if x=='high':
        return 1
    if x== 'med':
        return 2
    if x == 'low':
        return 3 

def Trans_Maint(x):
    
    if x== 'vhigh': 
        return 0
    if x=='high':
        return 1
    if x== 'med':
        return 2
    if x == 'low':
        return 3 

def Trans_doors(x):
    
    if x== '2': 
        return 0
    if x=='3':
        return 1
    if x== '4':
        return 2
    if x == '5more':
        return 3 

def Trans_persons(x):
    
    if x== '2': 
        return 0
    if x=='4':
        return 1
    if x== 'more':
        return 2
   
def Trans_lug_boot(x):
    
    if x== 'small': 
        return 0
    if x=='med':
        return 1
    if x== 'big':
        return 2
   

def Trans_safety(x):
    
    if x== 'low': 
        return 0
    if x=='med':
        return 1
    if x== 'high':
        return 2
   

def Trans_target(x):
    
    if x == 'unacc':
        return 0
    if x == 'acc':
        return 1
    if x == 'vgood':
        return 2
    if x == 'good':
        return 3

    
    
# let's use apply method to make the transformation    
  
data['Buying']=data['Buying'].apply(Trans_Buying)
data['Maint']=data['Maint'].apply(Trans_Maint)
data['doors']=data['doors'].apply(Trans_doors)
data['persons']=data['persons'].apply(Trans_persons)
data['lug-boot']=data['lug-boot'].apply(Trans_lug_boot)
data['safety']=data['safety'].apply(Trans_safety)
data['target'] = data['target'].apply(Trans_target)

In [17]:
#observe the data type here
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
Buying      1728 non-null int64
Maint       1728 non-null int64
doors       1728 non-null int64
persons     1728 non-null int64
lug-boot    1728 non-null int64
safety      1728 non-null int64
target      1728 non-null int64
dtypes: int64(7)
memory usage: 94.6 KB


In [18]:
data.head()

,Buying,Maint,doors,persons,lug-boot,safety,target
0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0
2,0,0,0,0,0,2,0
3,0,0,0,0,1,0,0
4,0,0,0,0,1,1,0


In [35]:
data.tail()

,Buying,Maint,doors,persons,lug-boot,safety,target
1723,3,3,3,2,1,1,3
1724,3,3,3,2,1,2,2
1725,3,3,3,2,2,0,0
1726,3,3,3,2,2,1,3
1727,3,3,3,2,2,2,2


# Building the model and traing

In [6]:
# let's split data nto training and testing
feature_columns = ['Buying', 'Maint','doors', 'persons','lug-boot','safety']

labels = data["target"].values
features = data[list(feature_columns)].values

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

### Random Forest Classifier

In [9]:
RFC = RandomForestClassifier(random_state=101)
RFC.fit(X_train, y_train)

print("Score on the training set is: {:2}"
      .format(RFC.score(X_train, y_train)))
print("Score on the test set is: {:.2}"
      .format(RFC.score(X_test, y_test)))

Score on the training set is: 0.9966914805624483
Score on the test set is: 0.95


### Decision Tree Classifier

In [10]:
DTC = tree.DecisionTreeClassifier(criterion = 'entropy')
DTC.fit(X_train, y_train)

print("Score on the training set is: {:2}"
      .format(DTC.score(X_train, y_train)))
print("Score on the test set is: {:.2}"
      .format(DTC.score(X_test, y_test)))

Score on the training set is: 1.0
Score on the test set is: 0.97


### Both models have a reall good performance, let's export the DTC model.

In [11]:
# Create persistent model
model_filename = 'carDTC.pkl'
print("Model exported to {}...".format(model_filename))
joblib.dump(DTC, model_filename)

Model exported to carDTC.pkl...


['carDTC.pkl']

# Now let's create a simple flask app for our API

**the following script must be saved into a .py file out of this notebook**

**Open a text editor follow up**

In [20]:
# importing the libraries
from flask import Flask, request, jsonify
from sklearn.externals import joblib

#Creating our FlaskAPP
app = Flask(__name__)

# Load the model
MODEL = joblib.load('carDTC.pkl')
MODEL_LABELS = ['unacc', 'acc', 'vgood', 'good']


# Creating the endpoint here called predict
- first it handles the input from the call.
- then making the input into one list.
- finally pass them to the model for prediction
- at the end we return to the suer the result in JSON

In [ ]:

@app.route('/predict')
def predict():
    # Retrieve query parameters related to this request.
    Buying = request.args.get('Buying')
    Maint = request.args.get('Maint')
    doors = request.args.get('doors')
    persons = requests.args.get('persons')
    lug_boot = request.args.get('lug_boot')
    safety = request.args.get('safety')

    # Our model expects a list of records
    features = [[Buying, Maint, doors, persons, lug_boot, safety]]


    # predict the class and probability of the class
    label_index = MODEL.predict(features)
    label_conf = MODEL.predict_proba(features)

    # Retrieve the name of the predicted class
    label = MODEL_LABELS[label_index[0]]

    # Create and send a response to the API caller
    return jsonify(status='complete', label=label,  label_conf = ''.join(str(label_conf)))



if __name__ == '__main__':
    app.run(debug=True)

# After you are done with scripting you API, fire it up by writing the followin command on your shell

python filename.py

**this will fire your application and it will be ready to recieve http requests so try the following request in your browser**

http://127.0.0.1:5000/predict?Buying=1&Maint=1&doors=1&persons=1&lug_boot=1&safety=1

Observe the response, try to change the values of each parameter and see how it makes different prediction from your model


In [37]:
import requests
response = requests.get('http://127.0.0.1:5000/predict?Buying=1&Maint=1&doors=1&persons=1&lug_boot=1&safety=1')
print(response.text)

{
  "Prediction": "Unacceptable", 
  "confidence": "[[1. 0. 0. 0.]]", 
  "probabilities": " Unacceptable = 1.0, Acceptable = 0.0, Very Good = 0.0, Good = 0.0", 
  "status": "complete"
}

